Chapite 1

Renseigner dans fileData le nom et le chemin de votre fichier skymap

In [ ]:
fileData = ''
import os.path
print(fileData)
if (os.path.exists(fileData)):
    print('Le fichier est présent')
else:
    print("Le fichier n'existe pas")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from astropy.io import fits
from astropy.wcs import WCS

controlPlot = True

Read data and find position of sources by modelling the spatial emision 

Première étape : ajustement du fond global

In [ ]:

file = fits.open(fileData)
image= file[0].data
cdelt = np.abs(file[0].header['CDELT1'])

x,y=np.mgrid[:image.shape[0],:image.shape[1]]
fit_p = fitting.LevMarLSQFitter()

modelBackground =  models.Gaussian2D(amplitude=15.2,x_mean=image.shape[0]/2,y_mean=image.shape[1]/2)

#Fit background
modelBackground = fit_p(modelBackground, x, y, image)
# Contol plot
if controlPlot:
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1, title='Données')
    plt.imshow(image)
    plt.subplot(1,3,2, title='Modèle')
    plt.imshow(modelBackground(x, y))
    plt.subplot(1,3,3, title='résidus (Donnée - Modèle)')
    plt.imshow(image-modelBackground(x, y))


Modèle pour Centaurus A

In [ ]:
modelSourceCenA = models.Gaussian2D(amplitude=15.2,x_mean=image.shape[0]/2,y_mean=image.shape[1]/2)

Modèle pour Source2

In [ ]:
modelSource2 = models.Gaussian2D(amplitude=15.2,x_mean=0.56*image.shape[0],y_mean=0.81*image.shape[1])

In [ ]:


imageModel = modelBackground+modelSourceCenA+modelSource2
#Make a symmetric gaussian
imageModel.y_stddev_1.tied = lambda model : model.x_stddev_1
imageModel.y_stddev_2.tied = lambda model : model.x_stddev_2
imageModel.theta_1.fixed = True
imageModel.theta_2.fixed = True

#Fit model with source       
imageModel = fit_p(imageModel, x, y, image, maxiter=500)
# Contol plot
if controlPlot:
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1, title='Données')
    plt.imshow(image)
    plt.subplot(1,3,2, title='Modèle')
    plt.imshow(imageModel(x, y))
    plt.subplot(1,3,3, title='résidus (Donnée - Modèle)')
    plt.imshow((image-imageModel(x, y)))

covMatrix = fit_p.fit_info['param_cov']
errVector = np.sqrt(covMatrix.diagonal())
    
coodSystem = WCS(fileData)

Dans imageModel, vous avez les differents paramètres des 3 gaussiennes : celle du fond, celle de Cen A et celle de source 2

Il y a 4 paramètres par gaussienne. Ceci permet de savoir quel paramètre correspond à quoi

In [ ]:
print(imageModel)

Voici comment est donnée la position de Cen A. En vous en inspirant, trouvez la position de Source 2

In [ ]:
#Determine position of Cen A
positionCenA = coodSystem.pixel_to_world(imageModel.y_mean_1.value, imageModel.x_mean_1.value)
errPositionCenA = np.mean([errVector[7],errVector[8]])*cdelt
sizeCenA = np.mean([imageModel.x_stddev_1.value,imageModel.y_stddev_1.value])*cdelt

print("Cen A: RA = "+str(positionCenA.ra.deg)+ " deg, Dec = " + str(positionCenA.dec.deg) + " deg")
print("Cen A: apparent size = " + str(sizeCenA) + " deg")
print("Cen A: Position uncertainty = "+str(errPositionCenA)+ " deg")